### Optimizando el uso de memoria
Cuando trabajamos con `pandas`, normalmente nuestro equipo no experimentará problemas de rendimiento mientras los datos que manejemos tengan un tamaño por debajo de los 100 megabytes. A partir de aquí los tiempos de ejecución se incrementan y nos exponemos a errores de **memoria insuficiente**.

Vamos a ver cómo optimizar el uso de la memoria poniendo a la hora de cargar nuestros datos

<img src="images/baseball.jpg">

### Cargando datos
Para ilustrar el caso manejaremos un fichero de datos procedente de [Retrosheet](http://www.retrosheet.org/gamelogs/index.html), priméramente cargaremos dicho fichero y veremos que aspecto tiene.

In [ ]:
import pandas as pd

In [ ]:
mg = pd.read_csv('datasets/MLB/mlb_games.csv', low_memory=True)

`pd.read_csv` proporciona multitud de parámetros, entre ellos `low_memory` que admite dos valores posibles,
* `True`, (valor por defecto) a la hora de cargar el fichero, en vez de procesarlo entero, lo hace por trozos. Esto implica un uso más óptimo de memoria en tiempo de carga (aunque también la posibilidad de "mixed types")
* `False`, determina el valor del tipo de dato una vez ha leído todas y cada una de las líneas del fichero evitando de esta manera conflictos de "mixed types" a costa de un mayor consumo de memoria en tiempo de carga.

Para más información se puede consultar https://github.com/pandas-dev/pandas/pull/13293#pullrequestreview-8216118

Revisamos el aspecto de nuestros datos, segúramente no comprendamos el significado de los campos. Para lo que se trata de ilustrar tampoco importa demasiado, no obstante se incluye un enlace al [diccionario de campos](http://www.retrosheet.org/gamelogs/glfields.txt)

In [ ]:
mg.head()

In [ ]:
pd.set_option('display.max_columns', None)

Es posible ajustar el display de nuestro notebook mediante una serie de opciones,
* pd.set_option('display.height', <value>)
* pd.set_option('display.max_rows', <value>)
* pd.set_option('display.max_columns', <value>)
* pd.set_option('display.width', <value>)


Ahora queremos ver el consumo de memoria que la carga de datos ha conllevado. Por defecto `pandas` nos da una aproximación, pero esta vez vamos a "forzar" para que nos dé una estimación más precisa.

In [ ]:
mg.info()

In [ ]:
mg.info(memory_usage='deep')

¡Vaya! son unos cuantos megas...vemos como `pandas` ha operado de la siguiente manera
* Para cualquier número entero utiliza el tipo `int64`.
* Para cualquier número en coma flotante utiliza el tipo `float64`.
* Para cualquier string, o cualquier campo cuyos valores presenten mezcla de tipos, utilizará el tipo `object`.

En sus [tripas](https://github.com/pandas-dev/pandas/blob/master/pandas/core/internals/blocks.py), `pandas` utiliza la clase `ObjectBlock` para representar aquellas columnas formadas por strings y tipos mixtos, la clase `IntBlock` para representar las formadas por enteros y para las columnas que contienen números en coma flotante, la clase `FloatBlock`

Pero lo que es más interesante, es que para todo lo que son números (sean enteros ó en coma flotante), `pandas` almacena sus valores como arrays de `numpy` (que como sabemos se basan en arrays de C) de tal manera que sus valores se almacenan en bloques contiguos de memoria. En pocas palabras, el acceso a este tipo de datos en `pandas` es muy ágil gracias a que se apoya en los `ndarray` de `numpy`.

Pero...¿qué tienen los `ndarray` para que sean eficientes?
* Los tipos de datos de las entradas del array.
* Un puntero a un bloque contiguo de memoria donde residen los datos del array.
* Una tupla que contenga la dimensión (shape) del array.
* Una tupla que contenga el stride (el tamaño del salto que hay que dar para moverse de un elemento a otro del array en las diferentes dimensiones del mismo).

Dado que cada tipo de dato se almacena por separado en memoria, revisaremos el consumo medio por tipo de columna

In [ ]:
for dtype in ['float','integer','object']:
    selected_dtype = mg.select_dtypes(include=[dtype])
    mean_usage_bytes = selected_dtype.memory_usage(deep=True)[1:].mean()
    mean_usage_mbytes = mean_usage_bytes / (1024**2)
    print("Uso medio de memoria para una columna de tipo {}: {:03.2f} MB".format(dtype,mean_usage_mbytes))

Pues sí...parece que lo menos óptimo de almacenar son los campos de tipo `object`

### Desgranando los tipos numéricos
Ya comentamos antes que `pandas` (entre bambalinas) almacena los tipos numéricos como `ndarrays` de `numpy` en bloques contiguos de memoria lo cual permite optimizar el acceso y el espacio requerido.

Ahora bien, hay que tener claro que principalmente los tipos numéricos (tanto `int` como `float`) vienen acompañados de sus propios subtipos que permiten ajustar mejor el número de bytes que se utilizará internamente para representar cada valor.
* Por ejemplo el tipo float comprende a su vez los subtipos `float16`, `float32` y `float64`, esto quiere decir que cada valor de cada uno de esos tipos ocupará 16 bits (2 bytes), 32 bits (4 bytes) y 64 bits (8 bytes) respectívamente.
* Para el tipo int, tendremos los subtipos `int8`, `int16`, `int32` e `int64`, con lo que cada valor representado por cada uno de ellos ocuparán 8 bits (1 byte), 16 bits (2 bytes), 32 bits (4 bytes) y 64 bits (8 bytes) respectívamente.

El tipo `uint` (unsigned integer) también tiene sus propios subtipos de manera similar al int: `uint8`, `uint16`, `uint32` y `uint64`

Pero de poco nos sirve esto si no tenemos claro los mínimos y máximos valores que son capaces de representar; esta información nos la pone al alcance los siguientes métodos
* Para números enteros `iinfo()`
* Para números en coma flotante `finfo()`

In [ ]:
import numpy as np

In [ ]:
sub_types = ["float64"]
for st in sub_types:
    print(np.finfo(st))

Ahora muestra tú los límites para el subtipo que prefieras.

### Optimizando los campos numéricos
Utilizaremos el método `to_numeric()` de `pandas` para hacer un downgrade de los tipos numéricos al subtipo más pequeño posible. Vamos a crear una función para calcular y luego poder comparar el uso de memoria.

In [ ]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_bytes = pandas_obj.memory_usage(deep=True).sum()
    else: # asumimos que si no es un DataFrame es una serie temporal (pandas Series)
        usage_bytes = pandas_obj.memory_usage(deep=True)
        
    usage_mbytes = usage_bytes / 1024 ** 2 # para convertir de bytes a megabytes
    return "{:03.2f} MB".format(usage_mbytes)

Si echamos un vistazo a las columnas interpretadas como `integer` vemos que se trata de `date`, `number_of_game`, `v_game_number`, `h_game_number`, `v_score` y `h_score`.

In [ ]:
mg.select_dtypes(include=['integer']).columns

En definitiva campos para los cuales un entero negativo no tendría sentido, es por ello que vamos a intentar hacer un downcast de los mismos a algún tipo de `unsigned`. Para ello utilizaremos la functión de `pandas`, `to_numeric`. Esta función hará lo siguiente,
* Convertir el argumento que le pasemos a tipo numérico.
* Si le pasamos algún valor al parámetro `downcast`, se intentará la conversión al subtipo más óptimo del tipo facilitado (ojo, esto solo ocurrirá si el parámetro ha podido ser convertido con éxito a tipo numérico o si ya era numérico).
* Solo se realizará la conversión indicada por el parámetro `downcast`, si el tipo resultante es de menor tamaño que el original.

In [ ]:
mg_integer = mg.select_dtypes(include=['integer'])
converted_uint = mg_integer.apply(pd.to_numeric,downcast='unsigned')

print("Before " + mem_usage(mg_integer))
print("After " + mem_usage(converted_uint))

Vemos que la mejora ha sido importante, aproximadamente un 80% de reducción del tamaño (aunque no es tan palpable en términos globales debido a las pocas features `integer` que tiene el dataset)

In [ ]:
# pd.concat nos permite concatenar dos objetos de pandas,
# a lo largo de un determinado eje que especificaremos con el
# parámetro axis.
compare_ints = pd.concat([mg_integer.dtypes,converted_uint.dtypes],axis=1)
compare_ints.columns = ['before','after']

In [ ]:
compare_ints

In [ ]:
compare_ints.apply(pd.Series.value_counts)

Repitamos la operación con nuestros tipos float

In [ ]:
mg_float = mg.select_dtypes(include=['float'])
converted_float = mg_float.apply(pd.to_numeric,downcast='float')

print("Before " + mem_usage(mg_float))
print("After " + mem_usage(converted_float))

In [ ]:
compare_floats = pd.concat([mg_float.dtypes, converted_float.dtypes], axis=1)
compare_floats.columns = ['before', 'after']
compare_floats.apply(pd.Series.value_counts)

Vemos que, mediante `pd.to_numeric()` y su parámetro `downcast`, se ha conseguido que los datos se ajusten a un tipo `float32`.

Ahora tenemos en `converted_int` y en `converted_float` las features (ó columnas) que hemos optimizado. Podemos crear una copia del DataFrame original y reemplazar sus columnas sin optimizar, por las que hemos optimizado.

In [ ]:
optimized_mg = mg.copy()

optimized_mg[converted_uint.columns] = converted_uint
optimized_mg[converted_float.columns] = converted_float

print("Tamaño original del dataset: " + mem_usage(mg))
print("Tamaño optimizado del dataset (tocando solo integer y float): " + mem_usage(optimized_mg))

No está mal, pero parece que la mayor optimización vendrá de encargarnos de aquellas features almacenadas como tipo `object`

### Desgranando los objetos

Veíamos como para los tipos numéricos, `pandas` se apoya en arrays de `numpy` (`ndarrays`) para optimizar su almacenamiento y por tanto el acceso a los mismos. Cuando hablamos de otros tipos como pueden ser los tipos `string`, numpy NO ofrece soporte para valores missing de esas características. Veamos esto con el comportamiento de la función de `numpy`, `isnan`:

In [ ]:
a1 = np.array([2.0, 3, -10])
np.isnan(a1)

In [ ]:
a2 = np.array([2.0, 3, -10, 'hola'])
np.isnan(a2)

A continuación vemos como se almacenan los tipos numéricos (que se apoyan en los arrays de numpy) en comparación con los tipos string. (gráfico procedente de este [post](https://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/))

<img src="images/storage.jpg">

Vemos que,
* En el caso de los tipos numéricos se trata de un único puntero a un buffer en el que están de forma contigua almacenados los valores.
* En el otro caso tenemos un puntero que a su vez apunta a buffer de punteros cada cual, a su vez, apunta al correspondiente objeto de Python que tiene la referencia al dato.

### ¿No hay manera de optimizar esto?

A partir de la versión 0.15, `pandas` introdujo los llamados categoricals (el nombre concreto del tipo es `category`) que nos permite,
* A partir de una feature ó columna que tenga un *número limitado de valores*.
* `pandas` crea por debajo un diccionario en el que cada valor de esa feature se mapeara con un número entero.
* Para ello utiliza el subtipo entero más eficiente posible capaz de representar todos los valores de esa feature.

<img src="images/category.jpg">

Parece buena idea pero...¿qué se quiere decir exactamente con un número *limitado* de valores?

In [ ]:
mg_object = mg.select_dtypes(include=['object']).copy()
mg_object.describe()

Teniendo en cuenta que nuestro dataset consta de unos 170000 registros, el número de valores únicos a simple vista es comparativamente bajo. Vamos a ver que pasaría si aplicamos un tipo `category` a, por ejemplo, `day_of_week`.

In [ ]:
dow = mg.day_of_week
print(dow.head())

Ahora haremos un casting o conversión de esta columna para que sea de tipo `category`

In [ ]:
dow_cat = dow.astype('category')
print(dow_cat.head())

Veamos los códigos que "por debajo" han sido asociados a cada categoría del tipo `category`

In [ ]:
dict(enumerate(dow_cat.cat.categories))

In [ ]:
dow_cat.head().cat.codes

En nuestro caso la feature `day_of_week` no tiene ningún valor missing, pero si así fuera el tipo `category` lo gestionaría asignándole el valor -1. Vamos a ver la ganancia de memoria que hayamos podido obtener tras la conversión.

In [ ]:
print(mem_usage(dow))
print(mem_usage(dow_cat))

Bastante impresionante, aunque
* Hay que tener en cuenta que este es un caso especialmente bueno (7 valores únicos y más de 170000 registros)
* Se pierde la posibilidad de hacer cálculos aritméticos y utilizar ciertos métodos como `min()` y `max()`

Una cuestión que seguro rondará la cabeza es...¿cuándo es realmente interesante recurrir a los tipos `category`? bueno, lo suyo es hacerlo cuando el número de valores únicos de una feature sea menor que el 50% del total...de lo contrario podríamos incurrir en un uso incluso mayor de memoria que si no usáramos los tipos `category`. Para saber más del tipo `category`, la documentación es de [ayuda](http://pandas.pydata.org/pandas-docs/stable/categorical.html#gotchas).

En base a esta regla que hemos establecido, comprobaremos para cada feature de tipo object si menos del 50% de sus valores son únicos y para las que cumplan la condición, se realizará la conversión a `category`.

In [ ]:
mg_object.head()

In [ ]:
converted_obj = pd.DataFrame()

for col in mg_object:
    num_unique_values = len(mg_object[col].unique())
    num_total_values = len(mg_object[col])
    
    if (num_unique_values / num_total_values) < 0.5:
        converted_obj[col] = mg_object[col].astype('category')
    else:
        converted_obj[col] = mg_object[col]

In [ ]:
print(mem_usage(mg_object))
print(mem_usage(converted_obj))

In [ ]:
compare_obj = pd.concat([mg_object.dtypes, converted_obj.dtypes], axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

In [ ]:
mg_object.v_league.value_counts()

In [ ]:
mg_object.h_league.value_counts()

Es momento de recopilar las mejores realizadas en los tipos numéricos y ver cual ha sido la optimización global con respecto a nuestra situación inicial.

In [ ]:
optimized_mg[converted_obj.columns] = converted_obj
mem_usage(optimized_mg)

Hemos pasado de 860.5 MB a 103.63 MB.
Pero antes de cerrar, recordemos que la primera columna de nuestro dataset era la feature `date` que a la hora de ejecutar el método `read_csv` al principio, fue considerada como `integer`.

In [ ]:
date = optimized_mg.date
print(mem_usage(date))
date.head()

Esta conversión no tiene mucho sentido y, aunque en términos de memoria nos salga más caro, lo coherente y útil a la hora de analizar nuestros datos será convertirlo a tipo `datetime`

In [ ]:
optimized_mg['date'] = pd.to_datetime(date,format='%Y%m%d')
print(mem_usage(optimized_mg))
optimized_mg.date.head()

### ¿Existe la posibilidad de seleccionar los tipos antes de crear el DataFrame?
Por suerte sí, ya que el método `pandas.read_csv` (`pd.read_csv` en nuestro caso, por el alias `pd` utilizado para referinos a `pandas`) tiene varios parámetros que nos permitirá informar a `pandas` del tipo de nuestras features.
Cabe destacar el parámetro `dtype` el cual es un diccionario cuyas claves son los nombres de las features y sus valores son tipos de `numpy`. Veamos en que consiste:

In [ ]:
# Puede parecer raro que prescindamos de la feature date,
# es simplemente para ver como pandas la gestionar por nosotros
optimized_types = optimized_mg.drop('date', axis=1).dtypes

In [ ]:
optimized_types

In [ ]:
optimized_types_col = optimized_types.index
optimized_types_type = [i.name for i in optimized_types.values]

Una vez tenemos por un lado dos listas, una con el nombre de las features y otra con su tipo óptimo. Preparamos el diccionario con ambas.

In [ ]:
column_dtypes = dict(zip(optimized_types_col, optimized_types_type))

In [ ]:
column_dtypes

Ahora cargaremos de nuevo el dataset con nuestro diccionario.

In [ ]:
read_optimized = pd.read_csv('datasets/MLB/mlb_games.csv', dtype=column_dtypes, parse_dates=['date'], infer_datetime_format=True)

In [ ]:
print(mem_usage(read_optimized))
read_optimized.head()

Es posible que para un dataset de este tamaño no parezca tan necesario el investigar sobre la tipología de nuestras features. Pero, como hemos visto, dejar que `pandas` infiera los tipos implica un mayor tiempo de carga (ya que hasta que no lea todo el fichero no determinará el tipo de una feature) y además, para evitar problemas, siempre tirará por aquellos subtipos con mayor precisión (64 bits).

Un uso más efectivo de `pandas` supondrá una mayor eficiencia en nuestras tareas cuando manejemos datasets de cierto tamaño.